# Model training

In this file, I want to build and train some models and select the best predictor of ace/no ace.

## Importing libraries

In [5]:
# Import everything from my_libraries.py
from my_libraries import *

# Other utilities
import tennis_utils
import dataframe_utils

In [6]:
# Read into dataframe
df = pd.read_csv("datasets/task_set.csv")

# Clean dataframe
df = dataframe_utils.clean_dataframe(df)

# Split data while maintaining the proportion of 'is_ace'
X = df.drop("is_ace", axis=1)
y = df["is_ace"]

# Perform stratified split using train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Save the train dataset in a new csv file
train_df = X_train
train_df["is_ace"] = y_train
train_df.to_csv("datasets/train_dataset.csv")

NameError: name 'SimpleImputer' is not defined